In [1]:

"""
Time Issues
-----------

The default time for wellbeing (April 2011 - March 2012 etc) appears in an odd alphabetical order on cmd.
Swtiching it YYYY QQ, eg April 2011 - March 2012 becomes: 2011 Q2 - 2012 Q1

{'July', 'October', 'January', 'April'} {'September', 'March', 'December', 'June'}
def switchTime(time):
    
    # convert months to quarters
    toQr = {
        'July':'Q3', 
        'October':'Q4', 
        'January':'Q1', 
        'April':'Q2',
        'September':'Q3',
        'March':'Q1',
        'December':'Q4',
        'June':'Q2'
    }
    
    firstyear = time.split("-")[0].strip().split(" ")[1]
    firstmonth = time.split("-")[0].strip().split(" ")[0]
    
    secondyear = time.split("-")[1].strip().split(" ")[1]
    secondmonth = time.split("-")[1].strip().split(" ")[0]
    
    return firstyear + " " + toQr[firstmonth] + " - " + secondyear + " " + toQr[secondmonth]
    
"""    
      

'\nTime Issues\n-----------\n\nThe default time for wellbeing (April 2011 - March 2012 etc) appears in an odd alphabetical order on cmd.\nSwtiching it YYYY QQ, eg April 2011 - March 2012 becomes: 2011 Q2 - 2012 Q1\n\n{\'July\', \'October\', \'January\', \'April\'} {\'September\', \'March\', \'December\', \'June\'}\ndef switchTime(time):\n    \n    # convert months to quarters\n    toQr = {\n        \'July\':\'Q3\', \n        \'October\':\'Q4\', \n        \'January\':\'Q1\', \n        \'April\':\'Q2\',\n        \'September\':\'Q3\',\n        \'March\':\'Q1\',\n        \'December\':\'Q4\',\n        \'June\':\'Q2\'\n    }\n    \n    firstyear = time.split("-")[0].strip().split(" ")[1]\n    firstmonth = time.split("-")[0].strip().split(" ")[0]\n    \n    secondyear = time.split("-")[1].strip().split(" ")[1]\n    secondmonth = time.split("-")[1].strip().split(" ")[0]\n    \n    return firstyear + " " + toQr[firstmonth] + " - " + secondyear + " " + toQr[secondmonth]\n    \n'

In [1]:
from databaker.framework import *
from databakerUtils.writers import v4Writer
import pandas as pd

tabs = loadxlstabs("PWB_estimates_Jul17_June18.xls")
fileOut = "LIVE_data_wellbeing_Jul17_June18.csv"

#tabs = loadxlstabs("personalwellbeingestimatesoctober2016toseptember20171.xls")
#fileOut = "TEST_data_wellbeing.csv"


# Skip the contents tab
tabs = [x for x in tabs if x.name.lower().strip() != "contents"]

conversionsegments = []
for tab in tabs:
    
    #############
    # ASSERTIONS
    # Throw a quick error is something isn't in the expected place.
    
    # Wellbeing sheets should always start with the period "April 2011 - March 2012"
    startDate = tab.filter("April 2011 - March 2012")
    assert len(startDate) == 1, "Cannot find the first date: 'April 2011 - March 2012'. Aborting."
    
    # Wellbeing sheets should always end with the footer "Source: Annual Population Survey, Office for National Statistics"
    footer = tab.filter("Source: Annual Population Survey, Office for National Statistics")
    assert len(footer) == 1, "Cannot find the footer: 'Source: Annual Population Survey, Office for National Statistics'. Aborting."
        
    # Must have a "Mean Average" cell
    assert len(tab.excel_ref('B').filter("Mean Average")) == 1, "Cannot find the 'Mean Average' cell in column B. Tab: " + tab.name
        
    if "uk" not in tab.name.lower():
        assert len(tab.excel_ref('B').filter("England")) == 1, "Cannot find cell with expected contents 'England'."
        
    #############
    # EXTRACTION
    
    obs = startDate.shift(RIGHT).expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
    obs = obs - footer.expand(RIGHT).expand(DOWN)
    
    time = startDate.expand(DOWN)
    
    estimate = tab.excel_ref('B').filter("Mean Average").shift(DOWN).expand(RIGHT).parent().is_not_blank().is_not_whitespace()
    
    wellbeingMeasure = tab.name.split("-")[1].strip()
    expectedMeasures = ["Life Satisfaction", "Happiness", "Worthwhile", "Anxiety"]
    assert wellbeingMeasure in expectedMeasures, "Cannot find measure in tab name. Expected one of:" + ",".join(expectedMeasures)
    
    
    """
    Re-mapping of estimate labels
    --------------------
    as-per Pauls conversation with business area

    Very good (9-10/ 0-1)
    Good (7-8/ 2-3)
    Fair (5-6/ 5-4)
    Poor (0-4/ 6-10)
    """
    newEstimateLabels = {
        "9 - 10":"Very good",
        "0 - 1":"Very good",
        "7 - 8":"Good",
        "2 - 3":"Good",
        "5 - 6":"Fair",
        "5 - 4":"Fair",
        "4 - 5":"Fair",
        "0 - 4":"Poor",
        "6 - 10":"Poor"
    }
    
    dimensions = [
        HDim(time, TIME, DIRECTLY, LEFT),
        HDim(estimate, "estimate", DIRECTLY, ABOVE, cellvalueoverride=newEstimateLabels),
        HDimConst("measureofwellbeing", wellbeingMeasure)
    ]
    
    # Append the relevent country/countries
    if "uk" in tab.name.lower():
        dimensions.append(HDimConst(GEOG, "K02000001"))
    else:
        codeLookup = {
            "England":"E92000001",
            "Wales":"W92000004",
            "Scotland":"S92000003",
            "Northern Ireland":"N92000002"
        }
        geog = tab.excel_ref('B').filter("England").expand(RIGHT).is_not_blank().is_not_whitespace()
        dimensions.append(HDim(geog, GEOG, CLOSEST, LEFT, cellvalueoverride=codeLookup))
    
    # Append the relevent country/countries
    conversionsegment = ConversionSegment(tab, dimensions, obs).topandas()
    conversionsegments.append(conversionsegment)


# Output to V4 but keep it in a dataframe for in-line post processing
outputData = pd.concat(conversionsegments)
v4Frame = v4Writer("", outputData, asFrame=True) 


# #################
# Post Proccessing


# Tidy ups
v4Frame["estimate"] = v4Frame["estimate"].map(lambda x: x.replace("Mean Average","Average (mean)"))


# Populate Geographic Place Names
codeLookup = {
            "K02000001":"United Kingdom",
            "E92000001":"England",
            "W92000004":"Wales",
            "S92000003":"Scotland",
            "N92000002":"Northern Ireland"
        }
for cl in codeLookup.keys():
    v4Frame["Geography"][v4Frame["Geography_codelist"] == cl] = codeLookup[cl]

    
# Chop time down to latter date, and make the codelist "Year Ending"
#v4Frame["Time"] = v4Frame["Time"].map(lambda x: x.split("-")[1].strip())
v4Frame["Time_codelist"] = "year ending"



# codelistify estimates and measureofwellbeing
def codelistify(cell):
    cell = cell.replace(" - ", "-").replace(" ", "-").lower()
    cell = cell.replace("(", "").replace(")","")
    return cell


v4Frame["estimate_codelist"] = v4Frame["estimate"].apply(codelistify)
v4Frame["measureofwellbeing_codelist"] = v4Frame["measureofwellbeing"].apply(codelistify)
    
    
# rename columns
reName = {
    "Time":"time",
    "Time_codelist":"year-ending",
    "Geography_codelist":"wellbeing-geography",
    "Geography":"geography",
    "measureofwellbeing":"allmeasuresofwellbeing"
}
newCols = []
for col in v4Frame.columns.values:
    if col in reName:
        newCols.append(reName[col])
    else:
        newCols.append(col)
v4Frame.columns = newCols

# BA opted to stck with alphabetical time
# v4Frame["time"] = v4Frame["time"].apply(switchTime)




v4Frame['time_codelist'] = v4Frame['time'].apply(lambda x:x.replace(' - ','-').replace(' ','-').lower())
v4Frame['geography_codelist'] = v4Frame['geography'].apply(lambda x:x.replace(' ','-').lower())

v4Frame.to_csv(fileOut, index=False)

Loading PWB_estimates_Jul17_June18.xls which has size 230912 bytes
Table names: ['Contents', 'UK - Life Satisfaction', 'UK - Worthwhile', 'UK - Happiness', 'UK - Anxiety', 'Country - Life Satisfaction', 'Country - Worthwhile', 'Country - Happiness', 'Country - Anxiety']
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
Extracting data structured as v4.


/Users/jamesbryant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
